##### This notebook loads the places data sourced from the FourSquare API
##### It uses the following columns from that data :
        POI
        Address
        City
        State
##### We use these columns as parameter to get a business match from yelp API to get an ID 
##### This information is used to get ratings and reviews for that business from the yelp reviews endpoint
##### This data is parsed and created into a new df1 and saved to FourSq.csv
    

In [39]:
import pandas as pd
import numpy as np
import json
from get_data_API import get_business_match
from get_data_API import get_reviews
import os
Yelp_key=os.getenv('Yelp_API')
Yelp_key_MR=os.getenv('Yelp_API(MR)')
print(Yelp_key)

6IU9EKTbYCfp6pBKhQLLhLrDcfIPWGDGKvLZsnU6TzdO2Oo5iipHNvTFsWwJvdKEV3MVsUd46gIG4nRhNd6IH3TmMj2figX0NI2hwFhd0NFgPmLQq47n--FyQo1uZXYx


In [33]:
get_reviews

<function get_data_API.get_reviews(id, API_key)>

In [10]:
foursq_df=pd.read_csv('FourSq_data.csv')
foursq_df.head()

,Venue,City,State,POI,Category,Address,Distance From Venue
0,Farm League Brewing,Cambridge,Ontario,NaN,NaN,NaN,NaN
1,Dickens Pub,Calgary,Alberta,NaN,NaN,NaN,NaN
2,Under the Neon Palms at the El Mocambo,Toronto,Ontario,The Grange,Museum,317 Dundas St W,759.0
3,Grand Bizarre Exhibition Place,Toronto,Ontario,NaN,NaN,NaN,NaN
4,The Starlite Room,Edmonton,Alberta,NaN,NaN,NaN,NaN


In [63]:
# map the states and make the changes in place in the data frame

state_mapping = {
    'Ontario': 'ON',
    'Alberta':'AB',
    'British Columbia':'BC',
    'Manitoba':'MB',
    'Saskatchewan':'SK',
    'New Brunswick':'NB',
    'Newfoundland and Labrador':'NL',
    'Prince Edward Island':'PEI',
    'Nova Scotia':'NS',
    'Quebec':'QC'
}

foursq_df['State'] = foursq_df['State'].map(state_mapping).fillna(foursq_df['State'])


In [64]:
# Filter rows where 'POI' column is not NaN
filtered_df = foursq_df[foursq_df['POI'].notna()]
filtered_df.head()
#filtered_df.shape

,Venue,City,State,POI,Category,Address,Distance From Venue
2,Under the Neon Palms at the El Mocambo,Toronto,ON,The Grange,Museum,317 Dundas St W,759.0
5,BBDB's,Vernon,BC,Vernon Museum,Museum,3009 32 Ave,223.0
6,BBDB's,Vernon,BC,Greater Vernon Museum & Archives,History Museum,3009 32 Ave,227.0
9,Enso Event Centre,Vancouver,BC,Chinese Cultural Centre of Greater Vancouver 大...,History Museum,50 Pender St E,657.0
10,Enso Event Centre,Vancouver,BC,Centre Stage,Science Museum,1455 Quebec St,556.0


#### pass the POI,city,state,address from the data frame and get yelp business id 

## Use get_business_match function to get business id on yelp to get the ratings for the POI above

In [74]:
business_id=[]
business_name=[]
cities=[]
for index, row in filtered_df.iterrows():
    name=row['POI']
    address=row['Address']
    city=row['City']
    state=row['State']
    res_fs = get_business_match(name,address,city,state,Yelp_key_yahoo)
    if res_fs.status_code == 200:
        data = res_fs.json()
        with open( f'business_fs.json','w') as f:
             json.dump(data,f)
       # Check if 'businesses' key exists in the JSON response and if it's not empty
        if 'businesses' in data and len(data['businesses']) > 0:
            business_id.append(data['businesses'][0]['id']) # only grab business id from the json
            business_name.append(data['businesses'][0]['name']) # only grab name from json
            cities.append(city)
        else:
            business_id.append('N/A')
            business_name.append(name)
            cities.append(city)
    else:
         print(f"Error: {res_fs.status_code}, {res_fs.text}")
         business_id.append('N/A')
         business_name.append(name)
         cities.append(city)

    print(city)

Toronto
Vernon
Vernon
Vancouver
Vancouver
Vancouver
Vancouver
Vancouver
Toronto
Error: 400, {"error": {"code": "VALIDATION_ERROR", "description": "\"The British Columbia Regiment - Duke of Connaugh t's Own Museum Society\" is too long", "field": "name", "instance": "The British Columbia Regiment - Duke of Connaugh t's Own Museum Society"}}
Vancouver
Vancouver
Kitchener
Kitchener
Kitchener
Montreal
Montreal
Hamilton
Hamilton
Hamilton
Hamilton
Hamilton
Niagara Falls
Niagara Falls
Niagara Falls
Kelowna
Kelowna
Toronto
Montreal
Montreal
London
London
Toronto
Toronto
Toronto
Toronto
St Catharines
Vancouver
Vancouver
Vancouver
Vancouver
Vancouver
Vancouver
Vancouver
Vancouver
Vancouver
Vancouver
Ottawa
Toronto
Toronto
Toronto
Toronto
Toronto
Toronto
Toronto
Toronto
Brossard
Brossard
Moncton
Toronto
Toronto
Error: 400, {"error": {"code": "VALIDATION_ERROR", "description": "'Office of the Information and Privacy Commissioner for British Columbia' is too long", "field": "name", "instance": "Off

In [76]:
Business_df1=pd.DataFrame({'business_id':business_id,
                        'business_name':business_name,
                          'city': cities})
Business_df1.to_csv('fs_business_id.csv',index=False)

In [78]:
Business_df1.drop_duplicates().shape

(143, 3)

In [77]:
Business_df1.shape

(154, 3)

## Use the business_id information to get ratings from get_reviews function 

In [83]:
ratings=[]
for index, row in Business_df1.iterrows():
    id=row['business_id']
    
    res_review=get_reviews(id,Yelp_key)
    print(index)
    if res_review.status_code == 200:
            review = res_review.json()
           # Check if 'reviews' key exists in the JSON response and if it's not empty
            if 'reviews' in review :
               ratings.append(review) # to keep list of reviews for each venue
                #dump data into json file
               with open(f'fs_ratings_yelp.json', 'w') as f: 
                 json.dump(ratings, f) 
            else:
                business_id.append('N/A')
                business_name.append(name)
    else:
             print(f"Error: {res_review.status_code}, {res_review.text}")
             ratings.append('N/A') # for venues with no business id available

print("Stopped at= ",index)

0
Error: 404, {"error": {"code": "NOT_FOUND", "description": "Resource could not be found."}}
1
2
Error: 404, {"error": {"code": "NOT_FOUND", "description": "Resource could not be found."}}
3
4
Error: 404, {"error": {"code": "NOT_FOUND", "description": "Resource could not be found."}}
5
6
Error: 404, {"error": {"code": "NOT_FOUND", "description": "Resource could not be found."}}
7
Error: 404, {"error": {"code": "NOT_FOUND", "description": "Resource could not be found."}}
8
Error: 404, {"error": {"code": "NOT_FOUND", "description": "Resource could not be found."}}
9
Error: 404, {"error": {"code": "NOT_FOUND", "description": "Resource could not be found."}}
10
Error: 404, {"error": {"code": "NOT_FOUND", "description": "Resource could not be found."}}
11
Error: 404, {"error": {"code": "NOT_FOUND", "description": "Resource could not be found."}}
12
Error: 404, {"error": {"code": "NOT_FOUND", "description": "Resource could not be found."}}
13
14
Error: 404, {"error": {"code": "NOT_FOUND", "

## Parse through the json fs_ratings_yelp2 to get a data frame of reviews and ratings details

In [84]:
with open ('fs_ratings_yelp.json','r') as f:
    fs_ratings=json.load(f)




In [85]:
review=[]
ratings=[]
venue_name=[]
venue_id=[]
city=[]

In [88]:
for index, row in Business_df1.iterrows():
       
    # Get the business ID from the current row
        if row['business_id'] != 'N/A':
    
        # Search for the corresponding rating in the ratings JSON data
            for review_item in fs_ratings:
    
                if 'reviews' in review_item :
                
                     for item in review_item['reviews']:
                     

                        if 'text' in item:
                            review.append(item['text'])
                        else:
                            review.append('N/A')

                        if 'rating' in item:
                            ratings.append(item['rating'])
                        else:
                            ratings.append('N/A')
                
                        venue_name.append(row.iloc[1])
                        venue_id.append(row.iloc[0])
                        city.append(row.iloc[2])
            
        else:
            review.append('N/A')
            ratings.append('N/A')
            venue_name.append(row.iloc[1])
            venue_id.append(row.iloc[0])
            city.append(row.iloc[2])

In [89]:
    
df1=pd.DataFrame({'venue':venue_name,
                 'venue_id':venue_id,
                  'city':city,
                 'reviews':review,
                 'ratings':ratings})
df1
    

,venue,venue_id,city,reviews,ratings
0,The Grange,N/A,Toronto,N/A,N/A
1,Museum,GE1FgoNrgxPk04ZSlfF7mA,Vernon,This is one of the worst museums I have ever v...,1
2,Museum,GE1FgoNrgxPk04ZSlfF7mA,Vernon,I didn't feel like there was anything special ...,2
3,Museum,GE1FgoNrgxPk04ZSlfF7mA,Vernon,This is suppose to be a tourist attraction pla...,1
4,Museum,GE1FgoNrgxPk04ZSlfF7mA,Vernon,I visited here for Tea Festival. The exhibitio...,2
...,...,...,...,...,...
1295,Maritime Museum of the Atlantic,Ei-bMnKvnPD4wiAlq0Jjeg,Halifax,"An excellent place to be, especially if you're...",5
1296,Maritime Museum of the Atlantic,Ei-bMnKvnPD4wiAlq0Jjeg,Halifax,Had the pleasure of visiting this museum durin...,5
1297,Maritime Museum of the Atlantic,Ei-bMnKvnPD4wiAlq0Jjeg,Halifax,This was a great stop on our trip. Lots to see...,5
1298,Maritime Museum of the Atlantic,Ei-bMnKvnPD4wiAlq0Jjeg,Halifax,I visited Halifax during a recent cruise. Whi...,5


In [92]:
df1.drop_duplicates(inplace=True)

In [93]:
df1.to_csv('FourSq_ratings.csv',index=False)